In [1]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec as wv
from tqdm import tqdm as tq

### Word2Vec을 이용해 비지도학습, 라벨링 진행

In [2]:
# 멜론에서 크롤링한 데이터 불러오기
data_dict = pd.read_csv('data_dic.csv')
lyr_result = data_dict['lyr_notEng_notBlack']

In [10]:
# 부족한 전처리 진행
a=''
for i in range(100):
    data_dict.drop(lyr_result[lyr_result=='…'+a].index, axis=0, inplace = True)
    a += ' …'

In [11]:
data_dict.drop(lyr_result[lyr_result=='…'].index, axis=0, inplace = True)

In [12]:
data_dict = pd.read_csv('data_dic.csv')
lyr_result = data_dict['lyr_notEng_notBlack']

In [13]:
data_dict.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,index,Unnamed: 0.1.1.1,issue_date,album_name,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,lyr_notEng,lyr_notEng_notBlack
0,0,0,0,10,10,20171221,BLUE,BLUE (Feat. AGER),GN0400,KIMM CHAAN,187,여기 서서 널 기다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 Yeah...,여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척...,여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척해...
1,1,1,1,11,11,20111215,Brown Eyed Soul Live Album `SOUL FEVER`,Gone,GN0400,브라운 아이드 소울,191,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...
2,2,2,2,12,12,20190318,시차증,시차증,GN0400,하루종일,200,love love love love love love love love 그땐 네가 ...,그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 ...,그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 그땐 으음 한땐...
3,3,3,3,16,16,20181222,"Menage a Trois：Sextape Vol. 1, 2, 3",Forever,GN1300,The-Dream,283,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...
4,4,4,4,17,17,20181122,Sun And Moon,그 여름밤,GN0400,샘김 (Sam Kim),289,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...


In [14]:
lyr_result[17]

'너와 난 둘로 이어져있어 난 알수 있어 모든 걸 다 잊구 눈빛은 당당해 자신감 가득해 스스로 내 자신에 꽤나 만족하지 여유가 충분해 나를 향한 모든 행동 어쩐지 너란 남자 맘에 들 것 같아 분명히 나를 갖고 싶은 욕심 있지만 신중히 다시 은근히 재고 있잖아 다가와 내 맘 내 눈 내 귀 모든 걸 바꿔봐 너 어디 나를 열어봐 워어우워어우워어어 하 괜한 흔한 그저 뻔한 호심이면 내 사랑 줄 수 없어 에 들어선 널 바라본 순간 느꼈어 루즈한 분에 사로잡혀 뭘 원하는 지도 바라는 지도 몰라 하지만은 다가와 내 어깨에 두 팔을 둘러줘 반쯤 감긴 눈에 살작 입술을 맞춰줘 못본척 스치고 안본척 숨고 난 그저 네 방식을 좀더 지켜 볼게 아닌 척 겉으론 다른 여자 대하면서 내쪽을 살피는 너 이젠 나를 잡아 가끔은 그냥 솔직하게 너를 보여봐 시간은 자꾸 가는데 너무 늦잖아 다가와 내 맘 내 눈 내 귀 모든 걸 바꿔봐 너 어디 나를 열어봐 워어우워어우워어어 하 괜한 흔한 그저 뻔한 호심이면 내 사랑 줄 수 없어 나에게 미소를 줘봐 너만의 느낌을 줘봐 찾아줘 지난 시간 내가 원한 걸 안겨줘 참았던 나를 깨워줘 워어우워어우워어어허 아픈 깊은 버리고픈 억 줄 거면 너 역시 그만 가줘 다가와 내 맘 내 눈 내 귀 모든 걸 바꿔봐 너 어디 나를 열어봐 워어우워어우워어어 하 괜한 흔한 그저 뻔한 호심이면 내 사랑 줄 수 없어 내가 고갤 돌린 틈을 타 점점 좁혀지는 사이 이대로 나 터져버릴 것만 같았지 나를 뚫어지게 본 네 맘을 다 알고 있어 그대 내게 다가와 이제는 간발의 차이 살짝 풍긴 그대의 향가 나는 좋아 나는 알 수 있어 이 밤의 그 끝을 잡아 다가와 줘'

In [15]:
# 형태소 분석기 mecab 사용
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

In [16]:
# 분류한 형태소 정수화하는 함수

#list_2d는 2차원 list이다. 2000개의 글과 각 글이 tokenized 된 단어들이다.
def word2num(list_2d):
    w2n_dic = dict()  # word가 key이고 index가 value인 dict
    n2w_dic = dict()  # index가 key이고 word가 value인 dict. 나중에 번호에서 단어로 쉽게 바꾸기 위해.
    idx = 1
    num_list = [[] for _ in range(len(list_2d))]   # 숫자에 매핑된 글의 리스트
    for k,i in enumerate(list_2d):
        if not i:
            continue
        elif isinstance(i, str): 
             # 내용이 단어 하나로 이루어진 경우, for loop으로 ['단어']가 '단'과 '어'로 나뉘지 않게 한다.
            if w2n_dic.get(i) is None:
                w2n_dic[i] = idx
                n2w_dic[idx] = i
                idx += 1
            num_list[k] = [dic[i]]
        else:
            for j in i:
                if w2n_dic.get(j) is None:
                    w2n_dic[j] = idx
                    n2w_dic[idx] = j
                    idx += 1
                num_list[k].append(w2n_dic[j])
    return num_list, w2n_dic, n2w_dic

In [17]:
# 토큰화 - 형태소 기반 (태깅 포함)
tagging_ex=[]
for i in tq(lyr_result):
    tag_word = mecab.pos(i)
    tagging_ex.append(tag_word)

100%|██████████████████████████████████████████████████████████████████████████| 12529/12529 [00:07<00:00, 1756.57it/s]


In [18]:
tagging_ex[581]

[('…', 'SY'), ('…', 'SE')]

In [19]:
# 붙어있는 태그(인덱스)로 불용어처리
stopword = []
for index in range(len(tagging_ex)):
    for word, tag in tagging_ex[index]:
        if tag in 'SF' or tag in 'SP' or tag in 'SY' or tag in 'SE' or
        tag in 'SS' or tag in 'SE' or tag in 'SO' or tag in 'SW' or
        tag in 'UN' or tag in 'OH'or tag in 'ON'or tag in 'OL'or tag in 'IC':
            stopword.append(word)

In [20]:
stopword

['뭐',
 '그',
 '으음',
 '으음',
 '그래',
 '으음',
 '으음',
 '으음',
 '으음',
 '아',
 '아',
 '그',
 '자',
 '오',
 '그',
 '참',
 '막',
 '아요',
 '막',
 '자',
 '막',
 '자',
 '뭐',
 '그래',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '에',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어허',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '네',
 '즐',
 '치',
 '그래',
 '얘',
 '안녕',
 '마',
 '아',
 '아',
 '그',
 '그',
 '그',
 '그',
 '아',
 '아',
 '뭐',
 '아냐',
 '그래',
 '아냐',
 '그래',
 '뭐',
 '얘',
 '얘',
 '아',
 '억',
 '그',
 '그',
 '억',
 '그',
 '그',
 '그',
 '그',
 '이야',
 '그',
 '이야',
 '그',
 '이야',
 '씨발',
 '아니요',
 '안녕',
 '썅',
 '그',
 '그래',
 '아',
 '그래',
 '그래',
 '뭐',
 '뭐',
 '어',
 '거',
 '어',
 '에이',
 '어',
 '아냐',
 '오',
 '오',
 '아냐',
 '오',
 '오',
 '네',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '그',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '야',
 '어쩜',
 '어쩜',
 '옳지',
 '옳지',
 '워',
 '옳지',
 '자',
 '음',
 '음',
 '어',
 '음',
 '음',
 '음',
 '음',
 '어',
 '음',
 '어',
 '음',
 '어',
 '오',
 '우',
 '우',
 '네',
 '네',
 '어',


In [21]:
tagging_ex[17]

[('너', 'NP'),
 ('와', 'JKB'),
 ('난', 'NP+JX'),
 ('둘', 'NR'),
 ('로', 'JKB'),
 ('이어져', 'VV+EC'),
 ('있', 'VX'),
 ('어', 'EC'),
 ('난', 'NP+JX'),
 ('알', 'VV+ETM'),
 ('수', 'NNB'),
 ('있', 'VV'),
 ('어', 'EC'),
 ('모든', 'MM'),
 ('걸', 'NNB+JKO'),
 ('다', 'MAG'),
 ('잊', 'VV'),
 ('구', 'EC'),
 ('눈빛', 'NNG'),
 ('은', 'JX'),
 ('당당', 'MAG'),
 ('해', 'VV+EC'),
 ('자신감', 'NNG'),
 ('가득', 'MAG'),
 ('해', 'VV+EC'),
 ('스스로', 'MAG'),
 ('내', 'MM'),
 ('자신', 'NNG'),
 ('에', 'JKB'),
 ('꽤', 'MAG'),
 ('나', 'JX'),
 ('만족', 'NNG'),
 ('하', 'XSV'),
 ('지', 'EC'),
 ('여유', 'NNG'),
 ('가', 'JKS'),
 ('충분', 'XR'),
 ('해', 'XSA+EC'),
 ('나', 'NP'),
 ('를', 'JKO'),
 ('향한', 'VV+ETM'),
 ('모든', 'MM'),
 ('행동', 'NNG'),
 ('어쩐지', 'MAG'),
 ('너', 'NP'),
 ('란', 'VCP+ETM'),
 ('남자', 'NNG'),
 ('맘', 'NNG'),
 ('에', 'JKB'),
 ('들', 'VV+ETM'),
 ('것', 'NNB'),
 ('같', 'VA'),
 ('아', 'EC'),
 ('분명히', 'MAG'),
 ('나', 'NP'),
 ('를', 'JKO'),
 ('갖', 'VV'),
 ('고', 'EC'),
 ('싶', 'VX'),
 ('은', 'ETM'),
 ('욕심', 'NNG'),
 ('있', 'VV'),
 ('지만', 'EC'),
 ('신중히', 'MAG'),
 ('다시', '

In [22]:
# 태그(인덱스) 없이 토큰화 - 형태소 기반
tagging_result=[]
for i in tq(lyr_result):
    tag_word = mecab.morphs(i)
    tag_word = [word for word in tag_word if not word in stopword] # stopword 사용
    tagging_result.append(tag_word)

num_list, w2n_dic, n2w_dic = word2num(tagging_result)

100%|████████████████████████████████████████████████████████████████████████████| 12529/12529 [15:39<00:00, 13.33it/s]


In [23]:
tagging_result[581]

[]

In [24]:
# 워드투벡터를 활용해 노래 가사를 학습하는 알고리즘
# batch_words -> 사용할 단어 개수 (나중에 딥러닝 모델 만들때 사용)
model_mecab = wv(tagging_result, vector_size = 150,min_count =1 
                 ,sg=0, batch_words=1000, window = 10)

In [25]:
model_mecab.wv.most_similar('이별')

[('아픔', 0.6502724289894104),
 ('미련', 0.6250076293945312),
 ('사랑', 0.6060248017311096),
 ('원망', 0.5918434262275696),
 ('후회', 0.5573021769523621),
 ('끝내', 0.5499873757362366),
 ('인연', 0.5421664118766785),
 ('상처', 0.5401149392127991),
 ('다림', 0.5386982560157776),
 ('헤어지', 0.5344676971435547)]

In [26]:
# word_vectors = model_mecab.wv 0626082926

# vacabs = word_vectors.key_to_index
# word_vectors_list = [word_vectors[v] for v in vacabs]

In [27]:
# 원하는 감정 사전 구축 알고리즘
lyric_emotion = ['아픔','다정','편안','아찔']
emotion_list = []
for index in range(len(lyric_emotion)):
    emotion_one=[] # 한개의 감정 단어가 담김
    for word in model_mecab.wv.most_similar(lyric_emotion[index], topn = 20):
        if(word[1] >= 0.5): # 유사도값 거리기반 0.5 이상인 단어들만
            emotion_one.append(word)
        emotion_list.append(emotion_one)

In [28]:
emotion_list

[[('슬픔', 0.8382664322853088),
  ('그리움', 0.7169186472892761),
  ('상처', 0.7081032395362854),
  ('눈물', 0.6685335636138916),
  ('고통', 0.6535555720329285),
  ('이별', 0.6502724885940552),
  ('아픈', 0.6496763229370117),
  ('다림', 0.6187857389450073),
  ('미움', 0.5990192294120789),
  ('추억', 0.5967745780944824),
  ('미련', 0.5915307998657227),
  ('세월', 0.5855915546417236),
  ('남겨준', 0.5816988945007324),
  ('슬픈', 0.5669152736663818),
  ('흔적', 0.5607787370681763),
  ('견뎌낼', 0.547825276851654),
  ('슬퍼하', 0.5406208038330078),
  ('절망', 0.536596417427063),
  ('지워', 0.5355479717254639),
  ('두려움', 0.5322193503379822)],
 [('슬픔', 0.8382664322853088),
  ('그리움', 0.7169186472892761),
  ('상처', 0.7081032395362854),
  ('눈물', 0.6685335636138916),
  ('고통', 0.6535555720329285),
  ('이별', 0.6502724885940552),
  ('아픈', 0.6496763229370117),
  ('다림', 0.6187857389450073),
  ('미움', 0.5990192294120789),
  ('추억', 0.5967745780944824),
  ('미련', 0.5915307998657227),
  ('세월', 0.5855915546417236),
  ('남겨준', 0.5816988945007324),
  ('

In [29]:
a = 0
for i in range(4):
    for k in range(19):
        emotion_list.pop(a)
    a += 1

In [30]:
emotion_list

[[('슬픔', 0.8382664322853088),
  ('그리움', 0.7169186472892761),
  ('상처', 0.7081032395362854),
  ('눈물', 0.6685335636138916),
  ('고통', 0.6535555720329285),
  ('이별', 0.6502724885940552),
  ('아픈', 0.6496763229370117),
  ('다림', 0.6187857389450073),
  ('미움', 0.5990192294120789),
  ('추억', 0.5967745780944824),
  ('미련', 0.5915307998657227),
  ('세월', 0.5855915546417236),
  ('남겨준', 0.5816988945007324),
  ('슬픈', 0.5669152736663818),
  ('흔적', 0.5607787370681763),
  ('견뎌낼', 0.547825276851654),
  ('슬퍼하', 0.5406208038330078),
  ('절망', 0.536596417427063),
  ('지워', 0.5355479717254639),
  ('두려움', 0.5322193503379822)],
 [('따듯', 0.7436991333961487),
  ('따스', 0.706627368927002),
  ('순수', 0.6776967644691467),
  ('향긋', 0.6640152931213379),
  ('촉촉', 0.6619295477867126),
  ('투명', 0.6599040627479553),
  ('포근', 0.6558176875114441),
  ('따뜻', 0.6554209589958191),
  ('달콤', 0.6513285040855408),
  ('화사', 0.6501985192298889),
  ('은은', 0.6425447463989258),
  ('야릇', 0.6241790056228638),
  ('수줍', 0.6233885884284973),
  ('섬세'

In [31]:
tagging_result[0]

['서',
 '서',
 '널',
 '다려',
 '걸어오',
 '는',
 '너',
 '는',
 '왜',
 '그리',
 '예쁜지',
 '괜히',
 '불안',
 '해지지',
 '나',
 '무심',
 '한',
 '척',
 '해도',
 '난',
 '나',
 '애써',
 '모른',
 '척',
 '해',
 '남자',
 '들',
 '너',
 '를',
 '탐내',
 '괜히',
 '억울',
 '해',
 '내겐',
 '배려',
 '도',
 '없',
 '게',
 '아름다운',
 '걸',
 '다',
 '너',
 '를',
 '바라볼',
 '때',
 '난',
 '너',
 '란',
 '별',
 '을',
 '향해',
 '환하',
 '게',
 '비춰',
 '줘',
 '계속',
 '내게만',
 '가',
 '나',
 '아닌',
 '하늘',
 '빛난다면',
 '난',
 '가',
 '너무',
 '예뻐서',
 '난',
 '불안',
 '해',
 '내',
 '맘',
 '같',
 '지',
 '않',
 '은',
 '걸',
 '넌',
 '마냥',
 '좋',
 '진',
 '않',
 '너',
 '가',
 '너무',
 '예쁜',
 '게',
 '남자',
 '라면',
 '한',
 '번',
 '쳐다볼',
 '걸',
 '너',
 '가',
 '지나갈',
 '때',
 '아마도',
 '머릿속',
 '에서',
 '계속',
 '너',
 '가',
 '생각',
 '나',
 '서',
 '잠',
 '을',
 '못',
 '이룰',
 '물론',
 '나',
 '는',
 '그렇',
 '지',
 '않',
 '오해',
 '지',
 '말',
 '줘',
 '너',
 '랑',
 '있',
 '을',
 '때',
 '면',
 '난',
 '어린아이',
 '가',
 '돼',
 '다른',
 '사람',
 '은',
 '절대',
 '모를',
 '아마도',
 '너',
 '를',
 '두',
 '고선',
 '누구',
 '도',
 '볼',
 '수',
 '없',
 '그러',
 '니',
 '내',
 '게',
 '줘',
 '가',
 '너무',
 '예뻐서',
 '난',
 '불안',


In [32]:
emotion_list[0][0]

('슬픔', 0.8382664322853088)

In [33]:
emotion_result = []
for z in range(len(tagging_result)):
    lyric_emotion = ['슬픔','순수','편안','아찔']
    data_test = tagging_result[z]
    # 계산용 Dictionary 구성
    data_dic = {lyric_emotion[0]:0,lyric_emotion[1]:0,lyric_emotion[2]:0,
               lyric_emotion[3]:0}

    for k in range(len(emotion_list)):
        for i in emotion_list[k]:
            for j in range(len(data_test)):
                if i[0] == data_test[j]:
                    data_dic[lyric_emotion[k]] = data_dic[lyric_emotion[k]]+i[1]
    #print(data_dic)
    def whatIsBest(x):
        return data_dic[x]

    best_emotion = max(data_dic.keys(), key = whatIsBest)
    print(best_emotion, data_dic[best_emotion])
    
    if data_dic[best_emotion] != 0:
        emotion_result.append(best_emotion)
        # 연관 단어가 있어서 점수가 나오면 가장 높은 감정을 리스트에 담음
    else:
        emotion_result.append('null') 
        # 연관 단어가 없으면 0으로 나옴. 리스트 입력 순서를 맞춰야 하기에 'null' 추가

순수 0.6039723753929138
편안 0.6240136623382568
슬픔 0
슬픔 0
편안 1.349590539932251
슬픔 0
슬픔 0
아찔 0.6807034015655518
순수 3.380707800388336
슬픔 0
슬픔 0
슬픔 0
슬픔 1.3048778176307678
슬픔 0
순수 0.6513285040855408
슬픔 0
슬픔 1.3370671272277832
슬픔 0.6496763229370117
슬픔 5.7290942668914795
슬픔 0.5967745780944824
슬픔 0
슬픔 0
슬픔 0
편안 0.6367999315261841
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 4.249181032180786
슬픔 0
슬픔 1.4162064790725708
슬픔 0
슬픔 5.258248448371887
슬픔 2.389305830001831
편안 1.93400377035141
순수 2.605314016342163
슬픔 0
슬픔 6.66707456111908
편안 2.574087381362915
슬픔 0
슬픔 1.265308141708374
슬픔 0
슬픔 3.922721743583679
슬픔 0.5669152736663818
슬픔 0
슬픔 0
순수 0.6513285040855408
슬픔 1.4162064790725708
슬픔 0.5967745780944824
슬픔 0
슬픔 0.7081032395362854
슬픔 2.619351029396057
슬픔 12.299216330051422
슬픔 0
슬픔 0
슬픔 0
편안 0.6240136623382568
순수 1.259393334388733
슬픔 0
편안 2.6819429993629456
순수 0.6513285040855408
슬픔 3.049906373023987
슬픔 1.3370671272277832
편안 1.349590539932251
슬픔 0
슬픔 0
슬픔 0.6502724885940552
슬픔 4.589118719100952
슬픔 1.0688157677650452


편안 0.6318351626396179
슬픔 0
슬픔 0
슬픔 0
순수 1.2467771768569946
슬픔 0
슬픔 0
슬픔 0
편안 0.6367999315261841
슬픔 3.9372461438179016
슬픔 0
슬픔 0
슬픔 0
슬픔 1.1935491561889648
슬픔 0
슬픔 0
슬픔 3.9564181566238403
슬픔 1.370485782623291
슬픔 0
슬픔 0
슬픔 1.2464509010314941
슬픔 0
슬픔 0
슬픔 0.7169186472892761
순수 1.413254737854004
슬픔 0.6502724885940552
편안 1.2636703252792358
슬픔 0
편안 0.6318351626396179
슬픔 0
슬픔 2.5147992968559265
슬픔 0
슬픔 0.5355479717254639
슬픔 0
슬픔 1.8872478008270264
슬픔 0
슬픔 0
편안 2.0579293370246887
슬픔 0
아찔 0.7281442880630493
순수 0.6233885884284973
슬픔 5.4311182498931885
슬픔 0
슬픔 0.5915307998657227
슬픔 0
슬픔 2.4693896770477295
슬픔 0
슬픔 2.0351285338401794
슬픔 0
순수 1.9539855122566223
슬픔 0
슬픔 1.2464509010314941
편안 0.6240136623382568
슬픔 0.6685335636138916
슬픔 0.6685335636138916
편안 0.6859764456748962
슬픔 0.6685335636138916
슬픔 0
순수 3.9423229694366455
순수 1.3067494630813599
순수 0.6513285040855408
순수 1.3026570081710815
편안 1.9465653896331787
슬픔 0.5855915546417236
슬픔 0
슬픔 2.005600690841675
슬픔 0
편안 2.0216113328933716
슬픔 1.337067127227

슬픔 0
슬픔 1.21031653881073
슬픔 0
슬픔 0.5967745780944824
슬픔 0
슬픔 0
순수 0.6513285040855408
슬픔 0
슬픔 0
순수 0.6132502555847168
슬픔 3.201601207256317
슬픔 1.7290971279144287
슬픔 0.6685335636138916
슬픔 0.5915307998657227
슬픔 0
순수 0.6132502555847168
슬픔 0
편안 0.6240136623382568
순수 0.7436991333961487
순수 1.2731543183326721
슬픔 0
슬픔 9.060982942581177
슬픔 1.1636898517608643
슬픔 0
슬픔 0.6685335636138916
슬픔 0
슬픔 0
슬픔 2.4091094732284546
슬픔 0
순수 1.3026570081710815
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 1.7417263984680176
순수 2.0198854207992554
아찔 2.0162415504455566
슬픔 0
슬픔 0
아찔 2.937077283859253
슬픔 0
슬픔 0
슬픔 2.451128602027893
슬픔 0
순수 3.256642520427704
슬픔 0
슬픔 0
슬픔 0
슬픔 1.936086356639862
슬픔 0
슬픔 0
슬픔 0
슬픔 1.1215574741363525
슬픔 0
슬픔 0
슬픔 0
슬픔 0
순수 1.3553935289382935
슬픔 0
슬픔 2.5481183528900146
슬픔 0
슬픔 5.314587354660034
슬픔 0.5669152736663818
슬픔 0
슬픔 1.3370671272277832
편안 0.6318351626396179
슬픔 0
슬픔 1.4338372945785522
슬픔 0
순수 1.3067494630813599
슬픔 0.7081032395362854
순수 1.8119171261787415
순수 0.6132502555847168
슬픔 1.7903237342834473
슬픔 0
슬픔 0


슬픔 0
순수 1.3300159573554993
슬픔 0
슬픔 1.6765328645706177
슬픔 2.384636104106903
슬픔 0
슬픔 0
슬픔 0
슬픔 0
편안 1.3004140257835388
슬픔 0
슬픔 0
편안 1.2993168830871582
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 1.8964086174964905
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6685335636138916
슬픔 1.8967233896255493
순수 1.362445056438446
순수 0.6159772872924805
아찔 0.7038286924362183
슬픔 0
슬픔 0.5322193503379822
슬픔 0
슬픔 0
슬픔 0.7081032395362854
슬픔 0
슬픔 1.4570521712303162
슬픔 0
슬픔 0
슬픔 0
아찔 0.6807034015655518
순수 1.8633657097816467
슬픔 0
슬픔 0
슬픔 0
슬픔 0
편안 1.226500391960144
슬픔 0.5990192294120789
슬픔 3.382237493991852
슬픔 3.440641701221466
순수 0.6513285040855408
슬픔 0
슬픔 0.6535555720329285
순수 0.6513285040855408
슬픔 3.0157995223999023
슬픔 0
슬픔 0
슬픔 0.7081032395362854
슬픔 0
슬픔 0
슬픔 2.977077841758728
슬픔 0.6685335636138916
슬픔 0.5915307998657227
슬픔 0
편안 0.6240136623382568
슬픔 0.5669152736663818
슬픔 0
슬픔 0
슬픔 0
편안 2.0579293370246887
슬픔 1.265308141708374
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5915307998657227
슬픔 0.7081032395362854
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 3.195405662059784
슬픔 0
슬픔 0
슬픔 1.299

슬픔 0
슬픔 0
슬픔 0
슬픔 0
순수 2.510601758956909
슬픔 0
편안 0.6859764456748962
슬픔 3.0556318759918213
슬픔 0.6685335636138916
슬픔 0
슬픔 1.3854522109031677
슬픔 0
편안 0.6318351626396179
순수 1.3331177234649658
슬픔 0
슬픔 0.6685335636138916
슬픔 0
순수 0.6513285040855408
슬픔 0
슬픔 1.5067999958992004
슬픔 2.5536587238311768
슬픔 2.619351029396057
슬픔 0.6685335636138916
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5967745780944824
슬픔 4.49969482421875
슬픔 0
슬픔 0
슬픔 4.679734945297241
순수 2.605314016342163
슬픔 2.005600690841675
슬픔 0
슬픔 1.1935491561889648
슬픔 0
편안 1.326411783695221
슬픔 0
슬픔 1.2993526458740234
슬픔 0
슬픔 0
아찔 0.710369884967804
슬픔 5.238702058792114
아찔 1.3309202194213867
슬픔 0.5967745780944824
슬픔 0
편안 2.013893723487854
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5355479717254639
슬픔 0
슬픔 0.6685335636138916
슬픔 0.536596417427063
슬픔 0
순수 1.3026570081710815
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 2.834576368331909
순수 1.2467771768569946
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6685335636138916
슬픔 0
슬픔 0
슬픔 0
슬픔 3.151229202747345
아찔 0.7878352403640747
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6502724885940552
슬픔 0
슬픔 3.9620100

...

슬픔 0.5355479717254639
슬픔 0.6685335636138916
슬픔 0
슬픔 0.5355479717254639
슬픔 0.6535555720329285
슬픔 0
순수 2.0092843770980835
슬픔 1.7903237342834473
슬픔 0.6685335636138916
슬픔 0
슬픔 3.5297205448150635
슬픔 5.2638572454452515
슬픔 7.052916884422302
슬픔 0
슬픔 1.8620827198028564
슬픔 0.7081032395362854
슬픔 4.166297495365143
슬픔 0
슬픔 0.7081032395362854
편안 0.6859764456748962
슬픔 0
슬픔 0
슬픔 4.851105093955994
순수 1.2467771768569946
슬픔 1.7903237342834473
슬픔 1.073192834854126
슬픔 0
편안 1.3004140257835388
슬픔 0
슬픔 1.6218624114990234
슬픔 0
슬픔 0
슬픔 0
순수 0.706627368927002
슬픔 5.181217610836029
슬픔 1.188305377960205
슬픔 0
순수 1.98893541097641
슬픔 0
슬픔 0
슬픔 0
슬픔 2.4879016280174255
슬픔 0
편안 1.3719528913497925
슬픔 3.3399423360824585
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
순수 0.7436991333961487
슬픔 0
슬픔 0
순수 1.3026570081710815
슬픔 0
슬픔 2.4024120569229126
편안 0.6859764456748962
슬픔 1.3671911358833313
슬픔 4.350667119026184
슬픔 5.008900165557861
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 1.8840938806533813
슬픔 0
슬픔 0
순수 0.6640152931213379
슬픔 0
순수 0.601000666618

슬픔 0
슬픔 0.6685335636138916
슬픔 0
슬픔 0
편안 0.6859764456748962
슬픔 1.9185582995414734
슬픔 0.7081032395362854
슬픔 0
편안 0.6636140942573547
슬픔 0
아찔 0.7281442880630493
순수 0.6513285040855408
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6502724885940552
슬픔 0
슬픔 2.8324129581451416
슬픔 0.6502724885940552
순수 1.3026570081710815
슬픔 0
슬픔 0.7081032395362854
슬픔 4.543597340583801
슬픔 2.005600690841675
슬픔 2.005600690841675
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5355479717254639
슬픔 1.1935491561889648
슬픔 0
슬픔 0
슬픔 0
슬픔 1.4338372945785522
슬픔 0
슬픔 0
편안 0.6367999315261841
슬픔 0
슬픔 0
슬픔 1.9508174657821655
슬픔 0.7081032395362854
슬픔 0
슬픔 0
슬픔 2.4405962228775024
슬픔 0
슬픔 0
아찔 0.7746283411979675
슬픔 0.6187857389450073
편안 1.3272281885147095
편안 2.510382056236267
슬픔 0
편안 0.6318351626396179
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 2.676585614681244
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 7.767430484294891
슬픔 0
슬픔 0
슬픔 1.1935491561889648
슬픔 1.3370671272277832
슬픔 0
슬픔 0
슬픔 1.9155806303024292
슬픔 0
순수 2.6310406923294067
슬픔 0.6685335636138916
슬픔 0.7081032395362854
아찔 0.6807034015655518
슬픔 0
슬픔 0

슬픔 0
슬픔 1.4162064790725708
슬픔 0
슬픔 0
순수 0.6039723753929138
슬픔 0
슬픔 0
순수 0.6425447463989258
슬픔 0
슬픔 3.251362442970276
슬픔 8.996315836906433
아찔 0.7217790484428406
슬픔 0
슬픔 0.7081032395362854
슬픔 2.494094133377075
슬픔 0
편안 0.6367999315261841
슬픔 0
슬픔 1.7207722663879395
슬픔 1.1935491561889648
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6496763229370117
슬픔 3.9513120651245117
슬픔 0
슬픔 1.2091543674468994
슬픔 1.8322234153747559
슬픔 0
슬픔 3.591540038585663
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.7169186472892761
편안 0.6859764456748962
슬픔 4.116195440292358
슬픔 0.6502724885940552
슬픔 0
슬픔 0
슬픔 2.005600690841675
슬픔 0
순수 3.2779109477996826
슬픔 0.5967745780944824
순수 0.6233885884284973
슬픔 1.1830615997314453
슬픔 0
슬픔 0
슬픔 0
슬픔 0.7169186472892761
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6685335636138916
슬픔 0
슬픔 0.8382664322853088
슬픔 0
슬픔 0.6187857389450073
슬픔 3.133879065513611
슬픔 0.6685335636138916
슬픔 0
슬픔 0
슬픔 0.5355479717254639
슬픔 0
슬픔 0
슬픔 0
슬픔 0
편안 1.2954492568969727
슬픔 0
슬픔 2.470897674560547
슬픔 3.187966823577881
순수 0.6513285040855408
슬픔 0.6685335636138916


슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6496763229370117
슬픔 0
슬픔 2.6741342544555664
슬픔 0
슬픔 0
슬픔 0.5355479717254639
편안 1.9908422827720642
슬픔 0
슬픔 0
순수 1.9539855122566223
슬픔 0
슬픔 0
슬픔 0.5967745780944824
슬픔 0.8382664322853088
슬픔 0.5355479717254639
슬픔 1.3370671272277832
슬픔 0.6535555720329285
슬픔 0
편안 0.6240136623382568
아찔 1.38453209400177
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
편안 0.6859764456748962
슬픔 0
편안 0.6859764456748962
아찔 2.8469711542129517
순수 1.3579558730125427
슬픔 0
슬픔 0
슬픔 3.222788453102112
순수 1.3290252685546875
슬픔 0
순수 0.706627368927002
슬픔 0
슬픔 0
슬픔 4.0997849106788635
아찔 0.702329158782959
슬픔 0
슬픔 0
슬픔 2.983872890472412
순수 1.2686712145805359
슬픔 1.4350410103797913
슬픔 0
슬픔 0
편안 4.115858674049377
순수 1.870165765285492
편안 0.6859764456748962
슬픔 1.9545541405677795
편안 1.283881664276123
슬픔 0
슬픔 0
슬픔 1.073192834854126
순수 2.826509475708008
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5355479717254639
슬픔 0.6685335636138916
편안 2.0579293370246887
슬픔 0.5967745780944824
슬픔 0
슬픔 0
슬픔 0
슬픔 0.5967745780944824
슬픔 0
슬픔 0
슬픔 0
순수 2.6053

슬픔 0
슬픔 0
슬픔 0.5322193503379822
슬픔 5.351316034793854
슬픔 0
슬픔 0.7081032395362854
슬픔 0
슬픔 0.5355479717254639
슬픔 0.7081032395362854
슬픔 0
순수 0.6513285040855408
편안 0.6859764456748962
슬픔 0
슬픔 0
슬픔 0
편안 1.2295461893081665
슬픔 8.814829349517822
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 1.0710959434509277
슬픔 0
아찔 0.6708096265792847
슬픔 0
슬픔 0
순수 1.3226046562194824
슬픔 0.5967745780944824
슬픔 0
슬픔 0
슬픔 0
슬픔 0.6496763229370117
슬픔 0
슬픔 0.6685335636138916
슬픔 0
아찔 0.6807034015655518
슬픔 0
슬픔 1.3577795624732971
슬픔 0
편안 1.3719528913497925
슬픔 0
슬픔 0
슬픔 7.0927422642707825
슬픔 0.6496763229370117
슬픔 0
슬픔 0
슬픔 1.4338372945785522
슬픔 0.5967745780944824
슬픔 0
슬픔 0.6685335636138916
슬픔 0.6502724885940552
슬픔 0
슬픔 5.5844115018844604
슬픔 0
슬픔 1.0644387006759644
슬픔 2.4046003818511963
슬픔 0
슬픔 0
슬픔 4.780878901481628
순수 0.6425447463989258
슬픔 0
슬픔 0.5967745780944824
슬픔 0
슬픔 0
슬픔 0
슬픔 0
슬픔 4.451574444770813
순수 0.6776967644691467
슬픔 0
슬픔 0.6685335636138916
슬픔 0
순수 0.6513285040855408
슬픔 0.5967745780944824
순수 0.6425447463989258
슬픔 0
슬픔 0
슬픔 4.7193046

In [34]:
data_temp =lyr_result
result_list = emotion_result

In [35]:
data_temp

0        여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척해...
1        또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...
2        그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 그땐 으음 한땐...
3        저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...
4        너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...
                               ...                        
12524    아무런 인사도 없이 우리 헤어졌지만 언젠가 또 다시 스쳐지나듯 만날꺼야 그때는 웃어...
12525    생각을 말아요 지나간 일들은 그리워 말아요 떠나간 님인데 꽃잎은 시들어요 슬퍼하지 ...
12526    그냥 날 지나쳐가줘 눈물 흘리지 않를 이 상황이 난 지겨워졌지 내가 바란 건 이런 ...
12527    행복한 이 설레임 느껴져 새롭게 또 시작된 날 쓰러진 내 삶을 힘겨운 내 맘을 이길...
12528    미러볼에 비치는 하늘과 그 이상과 어김없이 또 나는 그 곳에 가 있구나 너는 나를 ...
Name: lyr_notEng_notBlack, Length: 12529, dtype: object

In [36]:
result_pd = pd.Series(result_list)
data_label = pd.concat([data_temp, result_pd], axis = 1)
data_label

,lyr_notEng_notBlack,0
0,여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척해...,순수
1,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,편안
2,그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 그땐 으음 한땐...,null
3,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,null
4,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,편안
...,...,...
12524,아무런 인사도 없이 우리 헤어졌지만 언젠가 또 다시 스쳐지나듯 만날꺼야 그때는 웃어...,슬픔
12525,생각을 말아요 지나간 일들은 그리워 말아요 떠나간 님인데 꽃잎은 시들어요 슬퍼하지 ...,슬픔
12526,그냥 날 지나쳐가줘 눈물 흘리지 않를 이 상황이 난 지겨워졌지 내가 바란 건 이런 ...,슬픔
12527,행복한 이 설레임 느껴져 새롭게 또 시작된 날 쓰러진 내 삶을 힘겨운 내 맘을 이길...,null


In [37]:
#lyric_emotion = ['슬픔','순수','편안','아찔']
label_df = {'슬픔':0,'순수':1,'편안':2,'아찔':3}
data_label["label"] = data_label.iloc[:,1].map(label_df)
data_label

,lyr_notEng_notBlack,0,label
0,여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척해...,순수,1.0
1,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,편안,2.0
2,그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 그땐 으음 한땐...,null,NaN
3,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,null,NaN
4,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,편안,2.0
...,...,...,...
12524,아무런 인사도 없이 우리 헤어졌지만 언젠가 또 다시 스쳐지나듯 만날꺼야 그때는 웃어...,슬픔,0.0
12525,생각을 말아요 지나간 일들은 그리워 말아요 떠나간 님인데 꽃잎은 시들어요 슬퍼하지 ...,슬픔,0.0
12526,그냥 날 지나쳐가줘 눈물 흘리지 않를 이 상황이 난 지겨워졌지 내가 바란 건 이런 ...,슬픔,0.0
12527,행복한 이 설레임 느껴져 새롭게 또 시작된 날 쓰러진 내 삶을 힘겨운 내 맘을 이길...,null,NaN


In [38]:
data_label.shape

(12529, 3)

In [39]:
data_educate = data_label.drop(data_label[data_label.iloc[:,1] == 'null'].index, axis= 0)
data_educate = data_educate.dropna(axis=0) # null로 들어간값 제거

In [40]:
# 학습용 데이터 완료
data_educate.to_csv('educate_data.csv')

#### 모델 학습용 데이터 생성 완료

In [367]:
data_educate.columns=(1,2,'labels')
data_educate_result = data_educate.drop(2,axis=1)

### LSTM을 이용한 모델학습

In [369]:
# 여기까지가 원래 데이터(12560개)에 word2vec 이용해 라벨링 진행
# 다음부터는 라벨링이 된 데이터(6356개)를 다시 토큰화 -> 정수인코딩 -> 학습

In [370]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.datasets import imdb

import tensorflow as tf
import matplotlib.pyplot as plt

In [371]:
X = data_educate_result[1]
y = data_educate_result['labels']

In [372]:
X.shape, y.shape

((6460,), (6460,))

In [373]:
# 같은 방법으로 형태소 분리(토큰화)
X_ex=[]
for i in tq(X):
    tag_word = mecab.morphs(i)
    tag_word = [word for word in tag_word if not word in stopword] # 같은 불용어 처리
    X_ex.append(tag_word)

100%|██████████████████████████████████████████████████████████████████████████████| 6460/6460 [09:32<00:00, 11.28it/s]


In [374]:
len(X_ex)

6460

In [375]:
data_educate_result['tokenize']=X_ex
data_educate_result.to_csv('deeplearningFitItem.csv', index=False)

In [376]:
data_educate_result

,1,labels,tokenize
0,여 서서 널 다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 무심한 척해...,1.0,"[서, 서, 널, 다려, 걸어오, 는, 너, 는, 왜, 그리, 예쁜지, 괜히, 불안..."
1,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,0.0,"[또렷, 해져, 모두, 잊어버리, 려, 지워, 버리, 려, 할수록, 가득, 해져, ..."
4,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,2.0,"[너, 를, 처음, 만났, 던, 날, 밝, 게, 웃, 던, 게, 말, 걸, 었, 던..."
7,진하게 화장을 하고 예쁘게 머리를 하고 오늘도 집을 나서는 넌 예뻐 높은 구두를 신...,3.0,"[진하, 게, 화장, 을, 고, 예쁘, 게, 머리, 를, 고, 오늘, 도, 집, 을..."
8,바래다주고 오는 길 또 그새 보고 싶어서 내게 전화를 했어 나 참 안이랬었는데 너를...,2.0,"[바래다주, 고, 는, 길, 또, 그새, 보, 고, 싶, 어서, 내게, 전화, 를,..."
...,...,...,...
12522,바람이 차오 그만 가주오 이별을 선고 받은 마음 이미 죽은것과 같으니 눈물이 흐르오...,0.0,"[바람, 차오, 그만, 가, 주, 이별, 을, 선고, 받, 은, 마음, 이미, 죽,..."
12523,하루 종일 마시다 술에 취한 밤 그리워서 그리워서 또 이름 부른다 창밖을 바라보다 ...,0.0,"[하루, 종일, 마시, 다, 술, 취한, 밤, 그리워서, 그리워서, 또, 이름, 부..."
12524,아무런 인사도 없이 우리 헤어졌지만 언젠가 또 다시 스쳐지나듯 만날꺼야 그때는 웃어...,0.0,"[아무런, 인사, 도, 없이, 우리, 헤어졌, 지만, 언젠가, 또, 다시, 스쳐, ..."
12525,생각을 말아요 지나간 일들은 그리워 말아요 떠나간 님인데 꽃잎은 시들어요 슬퍼하지 ...,0.0,"[생각, 을, 말, 지나간, 일, 들, 은, 그리워, 말, 떠나간, 님, 인데, 꽃..."


In [377]:
# 모델 학습을 위해 위에서 지정해둔 함수로 정수화
num_list, w2n_dic, n2w_dic = word2num(X_ex)

In [378]:
num_list[0]

[1,
 1,
 2,
 3,
 4,
 5,
 6,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 13,
 19,
 20,
 16,
 21,
 22,
 23,
 6,
 24,
 25,
 10,
 26,
 21,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 6,
 24,
 35,
 36,
 18,
 6,
 37,
 38,
 39,
 40,
 41,
 31,
 42,
 43,
 44,
 45,
 46,
 13,
 47,
 48,
 49,
 18,
 46,
 50,
 51,
 18,
 11,
 21,
 52,
 53,
 54,
 55,
 56,
 57,
 33,
 58,
 59,
 60,
 61,
 56,
 6,
 46,
 50,
 62,
 31,
 22,
 63,
 15,
 64,
 65,
 33,
 6,
 46,
 66,
 36,
 67,
 68,
 69,
 44,
 6,
 46,
 70,
 13,
 1,
 71,
 39,
 72,
 73,
 74,
 13,
 5,
 75,
 55,
 56,
 76,
 55,
 77,
 43,
 6,
 78,
 79,
 39,
 36,
 80,
 18,
 81,
 46,
 82,
 83,
 84,
 57,
 85,
 86,
 67,
 6,
 24,
 87,
 88,
 89,
 29,
 90,
 91,
 30,
 92,
 93,
 52,
 31,
 43,
 46,
 50,
 51,
 18,
 11,
 21,
 52,
 53,
 54,
 55,
 56,
 57,
 33,
 58]

In [379]:
# from sklearn.model_selection import train_test_split
# x_tr, x_te, y_tr, y_te = train_test_split(X,y,test_size=0.2,random_state=7) 
# # 데이터들의 크기 확인하기
# x_tr.shape, x_te.shape, y_tr.shape, y_te.shape

In [380]:
import numpy as np

def divide_data(x, y, train_prop=0.8):
    x = np.array(num_list)
    y = np.array(y)
    tmp = np.random.permutation(np.arange(len(x)))
    x_tr = x[tmp][:round(train_prop * len(x))]
    y_tr = y[tmp][:round(train_prop * len(x))]
    x_te = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_te = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return x_tr, x_te, y_tr, y_te

In [381]:
x_tr, x_te, y_tr, y_te = divide_data(num_list, y)

<ipython-input-380-5120d1d642af>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(num_list)


In [382]:
x_tr.shape, x_te.shape, y_tr.shape, y_te.shape

((5168,), (1292,), (5168,), (1292,))

In [384]:
from keras.preprocessing import sequence
from keras.utils import np_utils
# 학습 데이터 크기 일정하게 지정
x_tr = sequence.pad_sequences(x_tr, maxlen=300) # 50개 넘으면 자르고 안되면 0으로 채움
x_te = sequence.pad_sequences(x_te, maxlen=300)

# 원핫 인코딩
y_tr = np_utils.to_categorical(y_tr) # one_hot으로 변형
y_te = np_utils.to_categorical(y_te) # 3 -> [0,0,0,1,0, ..]

In [385]:
y_tr

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [386]:
len(y_tr[0])

4

In [387]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping
# x_tr,x_te,y_tr,y_te
# words_num은 총 단어의 종류. +1을 해준 이유는 단어 수가 적은 글의 경우 빈 칸에 0이 있기 때문에.
model = Sequential()
model.add(Embedding(1000, len(x_tr[0])))  # 사용된 단어 수 & input 하나 당 size
model.add(LSTM(len(x_tr[0])))
model.add(Dense(len(y_tr[0]), activation='softmax'))  # 카테고리 수 (원핫해뒀기때문에 이대로)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es_callback = EarlyStopping(monitor = 'val_loss', patience=30)

In [388]:
import os
from keras.callbacks import ModelCheckpoint

# 모델을 생성할 폴더
MODEL_DIR = './model/'

# 폴더가 없다면 폴더 생성
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
# 저장할 파일명의 정규식을 설정(epoch수 - 손실값 을 파일명으로 설정)
modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}.hdf5'

# monitor = val_loss : 모니터링할 값(val_loss : 테스트오차, val_ass : 테스트 정확도)
# save_best_only : 이전에 저장한 모델보다 나아질경우 저장
checkpointer = ModelCheckpoint(filepath = modelpath,
               monitor='val_accuracy',verbose=1, save_best_only=True)

history = model.fit(x_tr, y_tr, batch_size=100, epochs=2000, 
                    callbacks=[es_callback, checkpointer], validation_data=(x_te,y_te),
                   )

Epoch 1/2000
52/52 [==============================] - 4s 58ms/step - loss: 1.1440 - accuracy: 0.5950 - val_loss: 1.0165 - val_accuracy: 0.6130

Epoch 00001: val_accuracy improved from -inf to 0.61300, saving model to ./model\01-1.0165.hdf5
Epoch 2/2000
52/52 [==============================] - 3s 53ms/step - loss: 0.9291 - accuracy: 0.6273 - val_loss: 0.8856 - val_accuracy: 0.6656

Epoch 00002: val_accuracy improved from 0.61300 to 0.66563, saving model to ./model\02-0.8856.hdf5
Epoch 3/2000
52/52 [==============================] - 3s 53ms/step - loss: 0.7935 - accuracy: 0.7109 - val_loss: 0.7783 - val_accuracy: 0.6935

Epoch 00003: val_accuracy improved from 0.66563 to 0.69350, saving model to ./model\03-0.7783.hdf5
Epoch 4/2000
52/52 [==============================] - 3s 53ms/step - loss: 0.7414 - accuracy: 0.7347 - val_loss: 0.8096 - val_accuracy: 0.7067

Epoch 00004: val_accuracy improved from 0.69350 to 0.70666, saving model to ./model\04-0.8096.hdf5
Epoch 5/2000
52/52 [===========

In [389]:
# validation 손실함수값, 정확도값 가장 합리적인 모델 불러오기
from tensorflow.keras.models import load_model
modelpath = MODEL_DIR + '26-0.6345.hdf5'
load_model = load_model(modelpath)

#21-1.2155.hdf5

In [390]:
# 로드한 모델 평가
load_model.evaluate(x_te,y_te)

41/41 [==============================] - ETA: 0s - loss: 0.6346 - accuracy: 0.79 - 1s 11ms/step - loss: 0.6345 - accuracy: 0.7957


[0.6344881057739258, 0.7956656217575073]

### LSTM 최적 모델로 예측, 평가 진행

In [391]:
#lyr_result = data_dic['lyr_notEng_notBlack']

In [392]:
# tagging_result=[]
# for i in lyr_result:
#     tag_word = mecab.morphs(i)
#     tagging_result.append(tag_word)

In [393]:
# num_list, w2n_dic, n2w_dic = word2num(tagging_result)

In [394]:
# a = model.predict(num_list[111])
# print(a[111])
# lyr_result[111]
# #'슬퍼','행복','불안','신나','편안','설렘','그리움','짜증(화남)'

In [395]:
# sad = 0
# happy = 0
# bulan= 0
# sinnam = 0
# rest= 0
# sullem = 0
# grium= 0
# anger= 0

# #lyric_emotion = ['슬픔','행복','불안','신나','편안','설렘','그리움','분노']
# lyric_emotion = {'sad':0,'happy':0,'bulan':0,
#                  'sinnam':0,'rest':0,'sullem':0,'grium':0,'anger':0}

In [396]:
# lyric_emotion['sad']

In [397]:
# for i in range(len(a)):
#     k = 0
#     for j in lyric_emotion:    
#         lyric_emotion[j] += a[i][k]
#         k +=1
# print(lyric_emotion)

In [398]:
# sorted(lyric_emotion.items(), key=operator.itemgetter(1))

In [399]:
stopword

['뭐',
 '그',
 '으음',
 '으음',
 '그래',
 '으음',
 '으음',
 '으음',
 '으음',
 '아',
 '아',
 '그',
 '자',
 '오',
 '그',
 '참',
 '막',
 '아요',
 '막',
 '자',
 '막',
 '자',
 '뭐',
 '그래',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '에',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어허',
 '워',
 '어우',
 '워',
 '어우',
 '워',
 '어',
 '어',
 '하',
 '네',
 '즐',
 '치',
 '그래',
 '얘',
 '안녕',
 '마',
 '아',
 '아',
 '그',
 '그',
 '그',
 '그',
 '아',
 '아',
 '뭐',
 '아냐',
 '그래',
 '아냐',
 '그래',
 '뭐',
 '얘',
 '얘',
 '아',
 '억',
 '그',
 '그',
 '억',
 '그',
 '그',
 '그',
 '그',
 '이야',
 '그',
 '이야',
 '그',
 '이야',
 '씨발',
 '아니요',
 '안녕',
 '썅',
 '그',
 '그래',
 '아',
 '그래',
 '그래',
 '뭐',
 '뭐',
 '어',
 '거',
 '어',
 '에이',
 '어',
 '아냐',
 '오',
 '오',
 '아냐',
 '오',
 '오',
 '네',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '그',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '야',
 '이야',
 '이야',
 '야',
 '어쩜',
 '어쩜',
 '옳지',
 '옳지',
 '워',
 '옳지',
 '자',
 '음',
 '음',
 '어',
 '음',
 '음',
 '음',
 '음',
 '어',
 '음',
 '어',
 '음',
 '어',
 '오',
 '우',
 '우',
 '네',
 '네',
 '어',


In [400]:
lyr_result.iloc[581]

'… …'

In [402]:
tagging_result[580]

['요',
 '지하철역',
 '문',
 '열리',
 '는',
 '순간',
 '환한',
 '빛',
 '나',
 '의',
 '눈',
 '을',
 '찔렀',
 '죠',
 '요',
 '그대',
 '가',
 '내게',
 '다가올',
 '때',
 '엔',
 '숨',
 '멎',
 '는',
 '것',
 '만',
 '같',
 '았',
 '죠',
 '요',
 '그대',
 '가',
 '내',
 '앞',
 '에서',
 '있',
 '을',
 '때',
 '난',
 '고갤',
 '들',
 '수',
 '조차',
 '없',
 '었',
 '는데',
 '요',
 '용',
 '를',
 '내',
 '말',
 '을',
 '건넬',
 '까',
 '자꾸',
 '생각',
 '만',
 '고',
 '있',
 '네요',
 '용가',
 '없',
 '어서',
 '난',
 '말',
 '도',
 '못',
 '꺼내',
 '고',
 '그대',
 '다리',
 '만',
 '한없이',
 '바라',
 '만',
 '보',
 '고',
 '있',
 '죠',
 '말',
 '한',
 '번',
 '걸',
 '볼까',
 '계속',
 '망설이',
 '다',
 '끝내',
 '말',
 '못',
 '죠',
 '내',
 '가',
 '너무',
 '자신',
 '없',
 '네요',
 '내',
 '가',
 '너무',
 '가진',
 '게',
 '없',
 '네요',
 '내',
 '가',
 '너무',
 '못나',
 '서',
 '그댄',
 '멀',
 '지',
 '네요',
 '요',
 '한',
 '번',
 '이라도',
 '말',
 '을',
 '건넬',
 '걸',
 '그대',
 '가',
 '떠난',
 '뒤',
 '후회',
 '되',
 '네요',
 '요',
 '머릿속',
 '에서',
 '계속',
 '떠올라',
 '이런',
 '내',
 '가',
 '너무',
 '한심',
 '해요',
 '요',
 '이름',
 '라도',
 '물',
 '봤',
 '으면',
 '전화',
 '번호',
 '라도',
 '물',
 '봤',
 '으면',
 '요',
 '다시',
 '만날',
 '수',
 

In [403]:
# 학습시킬 데이터 불러오기
data_dic = pd.read_csv('data_dic.csv')
lyr_result = data_dic['lyr_notEng_notBlack']

tagging_result=[]
for i in tq(lyr_result):
    tag_word = mecab.morphs(i)
    tag_word = [word for word in tag_word if not word in stopword] # 같은 불용어 처리
    tagging_result.append(tag_word)

100%|████████████████████████████████████████████████████████████████████████████| 12529/12529 [17:06<00:00, 12.21it/s]


In [431]:
num_list, w2n_dic, n2w_dic = word2num(tagging_result)

In [432]:
# 전처리 덜된애들 있음
nul_index = []
for i in range(len(num_list)):
    if not num_list[i]:
        nul_index.append(i)
        
for i in nul_index:
    del num_list[i]
    num_list.insert(0,'null')
    
for i in range(len(num_list)):
    if num_list[0]=='null':
        num_list.pop(0)

for i in range(len(num_list)):
    if not num_list[i]:
        print(i)

In [ ]:
#data_dict.drop(lyr_result[lyr_result==''].index, axis=0, inplace = True)

In [331]:
#196, 556
lyr_result[556]

'네가 무슨 말을 하든 안 들려 내가 말 안 해도 알겠지만 난 너만 진짜 너만 있으면 되는데 왜 모른 척 하는데 너도 알지 지금이 아니라면 이 순간을 놓친다면 우린 또 서롤 찾아 헤매게 될 텐데 네가 뭐를 두려워하는지 알아 내가 아니래도 못 믿겠지만 나는 진짜 나는 이런 적 없는데 너 땜에 자꾸 다른 사람이 돼 솔직해봐 지금이 아니라면 이 순간을 놓친다면 우린 또 서롤 찾아 헤매게 될 텐데 누가 뭐래든 너만 볼 거야 네가 어떻든 다 사랑할 거야 네 머릿속 걱정들 다 버리고 내 품 안에 쓰러져 대도 돼 꼭 안아줄게 그러니까'

In [ ]:
# 웹서비스 통합 구현 절차와 각 단계별 주요 언어활용방식을 학습할 수 있었습니다. 
# 특히 본작업은 백엔드, 프론트엔드 구분없이 기획이후 서비스 구현의 전 절차와 도구를 경험,
# 활용하였다는 점에서 교훈이 될 수 있었습니다. 
# 개발작업이 아닌 통합 구현을 만들어가는 전천우 개발자로서 경험을 활용하고자 합니다.

In [437]:
import operator

In [438]:
num_list[581]

[3059,
 151,
 6,
 13,
 1122,
 829,
 30,
 775,
 606,
 117,
 80,
 351,
 3024,
 54,
 1308,
 1535,
 747,
 169,
 757,
 112,
 192,
 145,
 589,
 23,
 39,
 545,
 114,
 60,
 57,
 55,
 1560,
 60,
 57,
 55,
 18,
 93,
 46,
 106,
 299,
 1215,
 2706,
 113,
 145,
 55,
 868,
 60,
 57,
 55,
 1629,
 169,
 6005,
 150,
 54,
 344,
 187,
 3803,
 759,
 6,
 424,
 2431,
 182,
 187,
 116,
 281,
 117,
 29,
 170,
 344,
 187,
 4590,
 759,
 6,
 112,
 1069,
 726,
 169,
 344,
 111,
 44,
 417,
 117,
 80,
 1992,
 278,
 616,
 145,
 36,
 70,
 13,
 120,
 1585,
 23,
 91,
 30,
 108,
 145,
 13,
 933,
 57,
 155,
 23,
 3563,
 219,
 745,
 188,
 420,
 1362,
 1724,
 6006,
 145,
 949,
 6007,
 15,
 5029,
 46,
 36,
 169,
 124,
 173,
 1528,
 54,
 57,
 808,
 46,
 139,
 29,
 2291,
 55,
 56,
 118,
 486,
 79,
 5,
 33,
 60,
 57,
 55,
 1560,
 60,
 57,
 55,
 18,
 93,
 46,
 106,
 299,
 1215,
 2706,
 113,
 145,
 55,
 868,
 60,
 57,
 55,
 1629,
 169,
 6005,
 150,
 54,
 344,
 187,
 3803,
 759,
 6,
 424,
 2431,
 182,
 187,
 116,
 281,
 117,
 29,

In [439]:
num_list[298]

[52,
 135,
 1282,
 79,
 5,
 2,
 90,
 36,
 80,
 18,
 496,
 288,
 353,
 47,
 150,
 371,
 18,
 4199,
 169,
 217,
 309,
 127,
 962,
 5,
 150,
 288,
 488,
 1610,
 151,
 33,
 106,
 117,
 312,
 808,
 858,
 5,
 6,
 24,
 90,
 36,
 828,
 13,
 5,
 496,
 745,
 6,
 103,
 281,
 5,
 124,
 39,
 90,
 91,
 79,
 109,
 31,
 4199,
 184,
 103,
 87,
 285,
 39,
 156,
 681,
 29,
 312,
 71,
 2046,
 5,
 150,
 583,
 46,
 4200,
 344,
 57,
 52,
 432,
 424,
 2,
 725,
 299,
 31,
 409,
 57,
 52,
 87,
 127,
 661,
 2,
 844,
 307,
 1436,
 1044,
 221,
 519,
 740,
 29,
 573,
 130,
 118,
 87,
 127,
 39,
 962,
 118,
 79,
 503,
 13,
 5,
 474,
 55,
 56,
 286,
 55,
 2626,
 15,
 244,
 221,
 4201,
 146,
 118,
 79,
 108,
 6,
 103,
 217,
 2715,
 151,
 33,
 18,
 192,
 745,
 5,
 85,
 146,
 55,
 56,
 39,
 31,
 238,
 150,
 23,
 447,
 151,
 33,
 344,
 57,
 52,
 432,
 424,
 2,
 725,
 299,
 31,
 409,
 57,
 52,
 87,
 127,
 661,
 2,
 844,
 307,
 1436,
 1947,
 108,
 145,
 106,
 58,
 4202,
 31,
 658]

In [513]:
# 만든 모델로 predict 진행
emotion1 = []
emotion2 = []
emotion_per = []

sad_sum = []
happy_sum = []
sexy_sum = []
rest_sum = []

for z in tq(range(len(num_list))):
    #label_df = {'슬픔':0,'순수':1,'편안':2,'아찔':3}
    #값을 뽑아내서 dictionary에 저장 -> list에 담기
    lyric_emotion = {'sad':0,'happy':0,'rest':0,'sexy':0}
    a = load_model.predict(num_list[z])
    for i in range(len(a)):
        k = 0
        for j in lyric_emotion:    
            lyric_emotion[j] += a[i][k]
            k +=1
            
    sad = lyric_emotion['sad']
    sad_sum.append(sad)
    happy = lyric_emotion['happy']
    happy_sum.append(happy)
    rest = lyric_emotion['rest']
    rest_sum.append(rest)
    sexy = lyric_emotion['sexy']
    sexy_sum.append(sexy)
    
    emo_per = sorted(lyric_emotion.items(), key=operator.itemgetter(1))
    emotion_per.append(emo_per)
    emo1 = emo_per[3][0]
    emotion1.append(emo1)
    emo2 = emo_per[2][0] 
    emotion2.append(emo2)


100%|████████████████████████████████████████████████████████████████████████████| 12394/12394 [09:09<00:00, 22.57it/s]


In [515]:
lyric_emotion['sad']

50.71109668185818

In [516]:
data_dic_emo = pd.read_csv('data_dic.csv')

In [517]:
data_dic_emo.drop(nul_index, axis = 0, inplace = True)

In [518]:
data_dic_emo.shape

(12394, 14)

In [519]:
data_dic_emo['emotion1'] = emotion1
data_dic_emo['emotion2'] = emotion2
data_dic_emo['emotion_per'] = emotion_per
data_dic_emo['sad_sum'] = sad_sum
data_dic_emo['happy_sum'] = happy_sum
data_dic_emo['rest_sum'] = rest_sum
data_dic_emo['sexy_sum'] = sexy_sum

In [520]:
data_dic_emo.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'index',
       'Unnamed: 0.1.1.1', 'issue_date', 'album_name', 'song_name',
       'song_gn_gnr_basket', 'artist_name_basket', 'id', 'lyric', 'lyr_notEng',
       'lyr_notEng_notBlack', 'emotion1', 'emotion2', 'emotion_per', 'sad_sum',
       'happy_sum', 'rest_sum', 'sexy_sum'],
      dtype='object')

In [521]:
data_dic_emo= data_dic_emo.drop(['Unnamed: 0.1.1.1','Unnamed: 0.1.1','Unnamed: 0.1','index','Unnamed: 0','lyr_notEng','lyr_notEng_notBlack',
                        'song_gn_gnr_basket'],
                        axis = 1)

In [522]:
data_dic_emo.head()

,issue_date,album_name,song_name,artist_name_basket,id,lyric,emotion1,emotion2,emotion_per,sad_sum,happy_sum,rest_sum,sexy_sum
0,20171221,BLUE,BLUE (Feat. AGER),KIMM CHAAN,187,여기 서서 널 기다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 Yeah...,rest,sad,"[(sexy, 27.080941318825353), (happy, 40.240577...",47.683238,40.240577,47.995243,27.080941
1,20111215,Brown Eyed Soul Live Album `SOUL FEVER`,Gone,브라운 아이드 소울,191,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,sad,rest,"[(sexy, 37.09707283275202), (happy, 47.8647014...",72.894142,47.864701,61.144084,37.097073
2,20190318,시차증,시차증,하루종일,200,love love love love love love love love 그땐 네가 ...,rest,sad,"[(sexy, 63.19080435251817), (happy, 71.4183282...",85.563479,71.418328,112.827389,63.190804
3,20181222,"Menage a Trois：Sextape Vol. 1, 2, 3",Forever,The-Dream,283,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,rest,sad,"[(happy, 36.153636154922424), (sexy, 47.559365...",67.996013,36.153636,78.290987,47.559365
4,20181122,Sun And Moon,그 여름밤,샘김 (Sam Kim),289,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,sad,rest,"[(sexy, 32.18946855748072), (happy, 33.4791203...",47.150453,33.479120,41.180957,32.189469


In [523]:
data_dic_emo.to_csv('korean_RnB.csv', index=False)

In [480]:
data_dic_emo.shape

(12394, 9)

In [481]:
# import pandas as pd
# data_dic = pd.read_csv('korean_RnB.csv')
# lyric_emotion = {'sad':0,'happy':0,'bulan':0,
#                  'sinnam':0,'rest':0,'sullem':0,'grium':0,'anger':0}
#    lyric_emotion = {'sad':0,'happy':0,'rest':0,'sexy':0}


In [483]:
a = []
for i in data_dic_emo['emotion1']:
    if i == 'sad':
        a.append('a')

len(a)

9048

In [495]:
data_dic_emo['emotion_per'][0]

[('sexy', 27.080941318825353),
 ('happy', 40.240577031858265),
 ('sad', 47.68323764693923),
 ('rest', 47.99524332024157)]

In [496]:
data_dic_emo['emotion_per'][0][0]

('sexy', 27.080941318825353)

In [518]:
data_dic

,issue_date,album_name,song_name,artist_name_basket,id,lyric,emotion1,emotion2,emotion_per
0,20171221,BLUE,BLUE (Feat. AGER),KIMM CHAAN,187,여기 서서 널 기다려 걸어오는 너는 왜 뭐 그리 예쁜지 괜히 불안해지지 나 Yeah...,sad,happy,"[(rest, 41.45942414691672), (sexy, 42.24034161..."
1,20111215,Brown Eyed Soul Live Album `SOUL FEVER`,Gone,브라운 아이드 소울,191,또렷해져 모두 잊어버리려 지워버리려 할수록 가득해져가는 너의 빈자리 지나온 날들 너...,happy,sad,"[(rest, 52.62763338990044), (sexy, 59.93924664..."
2,20190318,시차증,시차증,하루종일,200,love love love love love love love love 그땐 네가 ...,sexy,happy,"[(rest, 90.5323269850378), (sad, 93.0291958912..."
3,20181222,"Menage a Trois：Sextape Vol. 1, 2, 3",Forever,The-Dream,283,저 해와 달과 별 모두 빛을 잃고 세상의 구원자 죽으셨네 십자가에 달려 보혈 흘리셨...,rest,sad,"[(happy, 55.437779386295006), (sexy, 58.559330..."
4,20181122,Sun And Moon,그 여름밤,샘김 (Sam Kim),289,너를 처음 만났던 날 밝게 웃던 네게 말 걸었던 날 우리가 된 그 여름밤 가만히 내...,happy,sad,"[(sexy, 39.85448404410272), (rest, 42.35560599..."
...,...,...,...,...,...,...,...,...,...
12555,20080527,안녕,안녕,더 페이스,707747,아무런 인사도 없이 우리 헤어졌지만 언젠가 또 다시 스쳐지나듯 만날꺼야 그때는 웃어...,sad,sexy,"[(rest, 46.7146892844612), (happy, 48.82458362..."
12556,20010607,Brown Eyes,하얀나비,브라운 아이즈,707874,Um 생각을 말아요 지나간 일들은 Um 그리워 말아요 떠나간 님인데 꽃잎은 시들어요...,happy,rest,"[(sexy, 14.604965809354326), (sad, 15.56359101..."
12557,20191006,Insane and out,Lies (Feat. Lym en),K.vsh (캐시),707894,There is a lot of lies babe look what I got th...,sad,happy,"[(rest, 9.654458626406267), (sexy, 12.15483032..."
12558,20110902,I Decide (Piano Remix),I Decide (Piano Remix),헤리티지(Heritage),707969,And I will praise You 행복한 이 설레임 느껴져 And I love...,happy,sad,"[(rest, 12.676131755113602), (sexy, 12.9570687..."


In [522]:
data_dic[data_dic['emotion1'] == 'sexy']['lyric']

2        love love love love love love love love 그땐 네가 ...
6        Full of stars I wish I could be there with you...
16       행복했어 너와의 시간들 아마도 너는 힘들었겠지 너의 마음을 몰랐던 건 아니야 나도 ...
22       수많은 사람들 속에서 너를 봤어 유난히 니모습은 아름다워보여 널보면 알 수 없는 느...
24       매일 넌 말했지 서투른 표현에 제일 겁난댔지 첫 울음소리에 걱정 마 아마 우리 아빠...
                               ...                        
12531    달빛 아래 내 모습이 처량해 무거운 발걸음은 더 야속해 텅 빈 방안에 홀로 누워서 ...
12536    내가 아닌 것처럼 아무 일 없는 듯 난 낯선 오늘을 또 시작해 니가 없는 것처럼 오...
12542    왠지 어색한 표정 평소에 너답지 않은 달라진 옷차림 뭔가 어색하게 고개를 떨구고서 ...
12544    우리 다시 거기서 만날까 지금 나의 이 모습 그대로 먼지 쌓인 기억을 남기고 그넬 ...
12548    Call me cause I gon need a 이 밤은 길어 어제보다 더 nah ...
Name: lyric, Length: 3461, dtype: object

In [523]:
data_dic.iloc[2,5]

'love love love love love love love love 그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그땐 시간이 멈추던 그땐 으음 한땐 우리 사랑했던 한땐 내겐 무슨 말이 필요해 그땐 아무것도 필요 없던 그 땐 으음 24 hours I’m sick 그냥 멈춰져 있어 그대로 지구 어딘가엔 아마 that day that time 아직 흘러가고 있어 그래도 그래 이게 무슨 핑계가 돼 이렇게 너와 나의 공간은 서로 똑같은데 으음 난 다른 시간에 멈춰져 있어 그대로 oh 하루 또 하루를 replay 또 난 replay If somebody know tell me oh What’s the date today 며칠이 지나고 또 지난 지 알 수도 없이 밤을 걸어 난 이 시간이 그저 숫자에 지나지 않는 날을 지나 난 낮과 밤의 온도차를 모르는 날을 계속 보내 난 멀어져 한 번을 다 돌면 온전히 돌아오게 될까 127 degrees 같은 곳에 우린 서 있는데 네가 내게서 멀어질수록 시간이 더디게 가 그 격차는 점점 벌어지고 가는 길은 점점 멀어져 으음 난 난 다른 시간에 멈춰져있어 그대로 oh 매일 또 매일을 regret 또 난 regret If somebody know tell me oh What’s going on with me 며칠이 지나고 또 지난 지 알 수도 없이 밤을 걸어 난 이 시간이 그저 숫자에 지나지 않는 날을 지나 난 낮과 밤의 온도차를 모르는 날을 계속 보내 난 멀어져 한 번을 다 돌면 온전히 돌아오게 될까 아마 너의 시간은 계속 흘러가겠지만 어둔 밤 속에 혼자 이렇게 난 멈춰져있어 그대로 네가 멀어질수록 이 시차는 점점 커져 1hour plus 또 1hour 좁혀지지가 않아 며칠이 지나고 또 지난 지 알 수도 없이 밤을 걸어 난 이 시간이 그저 숫자에 지나지 않는 날을 지나 난 낮과 밤의 온도차를 모르는 날을 계속 보내 난 멀어져 한 번을 다 돌면 온전히 돌아오게 될까 그땐 네가 떠나가던 그땐 내겐 무슨 말이 필요해 그 때 시간이 멈추던